In [67]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [68]:
# Import the requests library.
import requests
# Import the API key
from config import weather_api_key

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [69]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats,lngs)
lat_lngs

In [70]:
from citipy import citipy

coordinates = list(lat_lngs)


cities = []
# Identify the nearest city for each lat and long combo.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

725

In [71]:
from datetime import datetime
city_data = []

record_count = 1
set_count = 1

In [75]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each d1city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Weather Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

rd 28 of Set 20 | dawei
Processing Record 29 of Set 20 | miranda
Processing Record 30 of Set 20 | naugarzan
City not found. Skipping...
Processing Record 31 of Set 20 | ust-uda
Processing Record 32 of Set 20 | boguchany
Processing Record 33 of Set 20 | katsuura
Processing Record 34 of Set 20 | nishihara
Processing Record 35 of Set 20 | lazaro cardenas
Processing Record 36 of Set 20 | mae hong son
Processing Record 37 of Set 20 | xingtai
Processing Record 38 of Set 20 | faanui
Processing Record 39 of Set 20 | trat
Processing Record 40 of Set 20 | te anau
Processing Record 41 of Set 20 | alice springs
Processing Record 42 of Set 20 | sao felix do xingu
Processing Record 43 of Set 20 | awbari
Processing Record 44 of Set 20 | bouillante
Processing Record 45 of Set 20 | pevek
Processing Record 46 of Set 20 | tigil
Processing Record 47 of Set 20 | richards bay
Processing Record 48 of Set 20 | thompson
Processing Record 49 of Set 20 | vila franca do campo
Processing Record 50 of Set 20 | gurs

In [76]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,Port Blair,11.6667,92.7500,82.69,82,100,21.09,IN,overcast clouds
1,Chuy,-33.6971,-53.4616,58.30,91,9,4.97,UY,light rain
2,Tuktoyaktuk,69.4541,-133.0374,59.00,47,75,14.97,CA,broken clouds
3,Tooele,40.5308,-112.2983,88.45,47,1,0.00,US,clear sky
4,Vaini,-21.2000,-175.2000,82.56,83,20,6.91,TO,few clouds
5,Lebu,-37.6167,-73.6500,48.78,82,100,8.55,CL,overcast clouds
6,Port Alfred,-33.5906,26.8910,50.11,68,41,6.33,ZA,scattered clouds
7,Ushuaia,-54.8000,-68.3000,37.06,75,90,9.22,AR,overcast clouds
8,Albany,42.6001,-73.9662,62.92,88,0,3.69,US,clear sky
9,Bethel,41.3712,-73.4140,69.58,85,25,3.76,US,scattered clouds


In [78]:
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Weather Description']
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Port Blair,IN,11.6667,92.7500,82.69,82,100,21.09,overcast clouds
1,Chuy,UY,-33.6971,-53.4616,58.30,91,9,4.97,light rain
2,Tuktoyaktuk,CA,69.4541,-133.0374,59.00,47,75,14.97,broken clouds
3,Tooele,US,40.5308,-112.2983,88.45,47,1,0.00,clear sky
4,Vaini,TO,-21.2000,-175.2000,82.56,83,20,6.91,few clouds


In [80]:
   # Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")